直接取回全部銀行指定匯率資訊

In [14]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Fetch the data
url = "https://www.findrate.tw/USD/"
response = requests.get(url)
data = response.text

# Parse the HTML
soup = BeautifulSoup(data, 'html.parser')
table = soup.find_all('table')[1]

# Extracting the information
exchange_rates = []
for row in table.find_all('tr')[1:]:  # Skip header row
    cols = row.find_all('td')
    if len(cols) == 7:  # Check if the row has the expected number of columns
        exchange_rates.append([col.text.strip() for col in cols])

# Create DataFrame
if exchange_rates:
    df = pd.DataFrame(exchange_rates, columns=["銀行名稱", "現鈔買入", "現鈔賣出", "即期買入", "即期賣出", "更新時間", "現鈔手續費"])
    # Save to Excel
    df.to_excel("usd_exchange_rates.xlsx", index=False)
else:
    print("No data found or table format is different.")


加入日期

In [18]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def fetch_data():
    # Fetch the data from the URL
    url = "https://www.findrate.tw/USD/"
    response = requests.get(url)
    data = response.text

    # Parse the HTML content
    soup = BeautifulSoup(data, 'html.parser')

    # Extract the date information
    # The date information is in the <span> tag with style="float:right"
    date_info = soup.find('span', style="float:right").text
    # Extracting the date in the format YYYY-MM-DD
    date = date_info.split('時間：')[-1].strip()

    # Find the table with the currency data
    table = soup.find_all('table')[1]  # The second table contains the required data

    # Extracting the information
    exchange_rates = []
    for row in table.find_all('tr')[1:]:  # Skip header row
        cols = row.find_all('td')
        if len(cols) == 7:  # Ensure the row has the expected number of columns
            row_data = [col.text.strip() for col in cols]
            # Remove HTML comments from the update time
            update_time = BeautifulSoup(row_data[5], "html.parser").text
            # Combine the date and the update time
            combined_datetime = f"{date} {update_time}"
            row_data[5] = combined_datetime.replace('-', '')
            exchange_rates.append(row_data)

    return exchange_rates

# Fetch data and create DataFrame
exchange_rates = fetch_data()
if exchange_rates:
    df = pd.DataFrame(exchange_rates, columns=["銀行名稱", "現鈔買入", "現鈔賣出", "即期買入", "即期賣出", "更新時間", "現鈔手續費"])
    # Save to Excel
    df.to_excel("usd_exchange_rates.xlsx", index=False)
else:
    print("No data found or table format is different.")

df.head()  # Display the first few rows of the DataFrame



,銀行名稱,現鈔買入,現鈔賣出,即期買入,即期賣出,更新時間,現鈔手續費
0,盤谷銀行,30.9,31.6,31.29,31.41,20231219 11:16,免手續費
1,星展銀行,31.095,31.596,31.295,31.396,20231219 11:16,"總額1%,最低NT$100"
2,板信銀行,31.095,31.595,31.295,31.395,20231219 11:16,"台幣總額0.5%,最低NT$100"
3,彰化銀行,31.045,31.615,31.295,31.395,20231219 11:16,結購外幣現鈔免收
4,永豐銀行,31.098,31.597,31.298,31.397,20231219 11:16,每筆NT$100


以下是從個別銀行取回指定資訊

In [1]:
from bs4 import BeautifulSoup
import requests

# Fetch the data from the URL
url = "https://www.findrate.tw/bank/10/"
response = requests.get(url)
data = response.text

# Parse the HTML content
soup = BeautifulSoup(data, 'html.parser')

# Find the table with the currency data
table = soup.find('table', {'width': '725px'})

# Extracting the USD information
usd_info = None
for row in table.find_all('tr'):
    cols = row.find_all('td')
    if cols and 'USD' in cols[0].text:
        usd_info = [col.text.strip() for col in cols]
        break

usd_info



['美金 USD', '30.982', '31.62', '31.297', '31.397']

加入銀行資訊

In [2]:
from bs4 import BeautifulSoup
import requests

# Fetch the data from the URL
url = "https://www.findrate.tw/bank/10/"
response = requests.get(url)
data = response.text

# Parse the HTML content
soup = BeautifulSoup(data, 'html.parser')

# Extracting the bank name
# 取得銀行名稱
bank_name = soup.find('div', {'id': 'Title'}).h1.text

# Extracting the bank transfer code and SWIFT code
# 取得跨行轉帳代碼和SWIFT銀行國際代碼
bank_info_paragraph = soup.find('div', {'id': 'Title'}).find_next_sibling('p')
bank_transfer_code = bank_info_paragraph.find_all('b')[0].text  # 跨行轉帳代碼
swift_code = bank_info_paragraph.find_all('b')[1].text         # SWIFT銀行國際代碼

# Find the table with the currency data
table = soup.find('table', {'width': '725px'})

# Extracting the USD information
usd_info = None
for row in table.find_all('tr'):
    cols = row.find_all('td')
    if cols and 'USD' in cols[0].text:
        usd_info = [col.text.strip() for col in cols]
        break

# Output the extracted information
print("Bank Name:", bank_name)
print("Bank Transfer Code:", bank_transfer_code)
print("SWIFT Code:", swift_code)
print("USD Information:", usd_info)


Bank Name: 合作金庫牌告匯率表
Bank Transfer Code: 006
SWIFT Code: TACBTWTP
USD Information: ['美金 USD', '30.982', '31.62', '31.297', '31.397']


In [3]:
from bs4 import BeautifulSoup
import requests

# Fetch the data from the URL
url = "https://www.findrate.tw/bank/9/"
response = requests.get(url)
data = response.text

# Parse the HTML content
soup = BeautifulSoup(data, 'html.parser')

# Extracting the bank name
# 取得銀行名稱
bank_name = soup.find('div', {'id': 'Title'}).h1.text

# Extracting the bank transfer code and SWIFT code
# 取得跨行轉帳代碼和SWIFT銀行國際代碼
bank_info_paragraph = soup.find('div', {'id': 'Title'}).find_next_sibling('p')
bank_transfer_code = bank_info_paragraph.find_all('b')[0].text  # 跨行轉帳代碼
swift_code = bank_info_paragraph.find_all('b')[1].text         # SWIFT銀行國際代碼

# Find the table with the currency data
table = soup.find('table', {'width': '725px'})

# Extracting the USD information
usd_info = None
for row in table.find_all('tr'):
    cols = row.find_all('td')
    if cols and 'USD' in cols[0].text:
        usd_info = [col.text.strip() for col in cols]
        break

# Output the extracted information
print("Bank Name:", bank_name)
print("Bank Transfer Code:", bank_transfer_code)
print("SWIFT Code:", swift_code)
print("USD Information:", usd_info)


Bank Name: 台新銀行牌告匯率表
Bank Transfer Code: 812
SWIFT Code: TSIBTWTP
USD Information: ['美金 USD', '31.105', '31.582', '31.307', '31.407']


存檔案

In [6]:
import pandas as pd
import requests

# Fetch the data from the URL
url = "https://www.findrate.tw/bank/9/"
response = requests.get(url)
data = response.text

# Parse the HTML content
soup = BeautifulSoup(data, 'html.parser')

# Extracting the bank name
# 取得銀行名稱
bank_name = soup.find('div', {'id': 'Title'}).h1.text

# Extracting the bank transfer code and SWIFT code
# 取得跨行轉帳代碼和SWIFT銀行國際代碼
bank_info_paragraph = soup.find('div', {'id': 'Title'}).find_next_sibling('p')
bank_transfer_code = bank_info_paragraph.find_all('b')[0].text  # 跨行轉帳代碼
swift_code = bank_info_paragraph.find_all('b')[1].text         # SWIFT銀行國際代碼

# Find the table with the currency data
table = soup.find('table', {'width': '725px'})

# Extracting the USD information
usd_info = None
for row in table.find_all('tr'):
    cols = row.find_all('td')
    if cols and 'USD' in cols[0].text:
        usd_info = [col.text.strip() for col in cols]
        break

# Output the extracted information
print("Bank Name:", bank_name)
print("Bank Transfer Code:", bank_transfer_code)
print("SWIFT Code:", swift_code)
print("USD Information:", usd_info)


Bank Name: 台新銀行牌告匯率表
Bank Transfer Code: 812
SWIFT Code: TSIBTWTP
USD Information: ['美金 USD', '31.106', '31.583', '31.308', '31.408']


查詢多家

In [7]:
import pandas as pd
import requests


def fetch_data(_index):
    # Fetch the data from the URL
    url = f"https://www.findrate.tw/bank/{_index}/"
    response = requests.get(url)
    data = response.text

    # Parse the HTML content
    soup = BeautifulSoup(data, 'html.parser')

    # Extracting the bank name
    # 取得銀行名稱
    bank_name = soup.find('div', {'id': 'Title'}).h1.text

    # Extracting the bank transfer code and SWIFT code
    # 取得跨行轉帳代碼和SWIFT銀行國際代碼
    bank_info_paragraph = soup.find('div', {'id': 'Title'}).find_next_sibling('p')
    bank_transfer_code = bank_info_paragraph.find_all('b')[0].text  # 跨行轉帳代碼
    swift_code = bank_info_paragraph.find_all('b')[1].text         # SWIFT銀行國際代碼

    # Find the table with the currency data
    table = soup.find('table', {'width': '725px'})

    # Extracting the USD information
    usd_info = None
    for row in table.find_all('tr'):
        cols = row.find_all('td')
        if cols and 'USD' in cols[0].text:
            usd_info = [col.text.strip() for col in cols]
            break

    # Output the extracted information
    print("Bank Name:", bank_name)
    print("Bank Transfer Code:", bank_transfer_code)
    print("SWIFT Code:", swift_code)
    print("USD Information:", usd_info)


if __name__ == '__main__':
    for i in range(1, 10):
        fetch_data(i)
        print("=" * 50)

Bank Name: 上海商銀牌告匯率表
Bank Transfer Code: 011
SWIFT Code: SCSBTWTP
USD Information: ['美金 USD', '31.09', '31.59', '31.29', '31.4']
Bank Name: 中國信託牌告匯率表
Bank Transfer Code: 822
SWIFT Code: CTCBTWTP
USD Information: None
Bank Name: 京城銀行牌告匯率表
Bank Transfer Code: 054
SWIFT Code: TNBBTWTN
USD Information: ['美金 USD', '30.927', '31.617', '31.297', '31.397']
Bank Name: 元大銀行牌告匯率表
Bank Transfer Code: 806
SWIFT Code: APBKTWTH
USD Information: ['美金 USD', '31.046', '31.606', '31.296', '31.396']
Bank Name: 兆豐銀行牌告匯率表
Bank Transfer Code: 017
SWIFT Code: ICBCTWTP
USD Information: ['美金 USD', '30.95', '31.62', '31.29', '31.39']
Bank Name: 匯豐銀行牌告匯率表
Bank Transfer Code: 081
SWIFT Code: HSBCTWTP
USD Information: ['美金 USD', '31.0445', '31.6445', '31.2945', '31.3945']
Bank Name: 台中銀行牌告匯率表
Bank Transfer Code: 053
SWIFT Code: TCBBTWTH
USD Information: ['美金 USD', '31.044', '31.594', '31.294', '31.394']
Bank Name: 富邦銀行牌告匯率表
Bank Transfer Code: 012
SWIFT Code: TPBKTWTP
USD Information: ['美金 USD', '31.139', '31.549',

存檔

In [11]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

def fetch_data(_index):
    # Fetch the data from the URL
    url = f"https://www.findrate.tw/bank/{_index}/"
    response = requests.get(url)
    data = response.text

    # Parse the HTML content
    soup = BeautifulSoup(data, 'html.parser')

    full_bank_name = soup.find('div', {'id': 'Title'}).h1.text
    # Extracting the bank name
    bank_name = full_bank_name.split('牌告匯率')[0]

    # Extracting the bank transfer code and SWIFT code
    bank_info_paragraph = soup.find('div', {'id': 'Title'}).find_next_sibling('p')
    bank_transfer_code = bank_info_paragraph.find_all('b')[0].text
    swift_code = bank_info_paragraph.find_all('b')[1].text

    # Find the table with the currency data
    table = soup.find('table', {'width': '725px'})

    # Extracting the USD information
    usd_info = None
    for row in table.find_all('tr'):
        cols = row.find_all('td')
        if cols and 'USD' in cols[0].text:
            usd_info = [col.text.strip() for col in cols[1:]]  # Extract only the rate values
            break

    # Check if USD info is found
    if usd_info is not None:
        return [bank_name, bank_transfer_code, swift_code] + usd_info
    else:
        # Return None for missing USD info fields
        return [bank_name, bank_transfer_code, swift_code] + [None, None, None, None]


# Prepare to store the data
all_bank_data = []

# Fetch data for each bank
for i in range(1, 10):
    all_bank_data.append(fetch_data(i))

# Create a DataFrame from the data
df = pd.DataFrame(all_bank_data, columns=[
    "銀行名稱", "銀行代碼", "SWIFT Code", "現金買入", "現金賣出", "即期買入", "即期賣出"
])

# Save the DataFrame to an Excel file
df.to_excel("exchange_rates.xlsx", index=False)
